In [46]:
!pip install Folium
!pip install geocoder
!pip install shapely
import shapely.geometry

!pip install pyproj
import pyproj

import math

     |████████████████████████████████| 1.8MB 6.7MB/s eta 0:00:01
     |████████████████████████████████| 10.9MB 6.5MB/s eta 0:00:01


In [4]:

import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

from IPython.display import Image 
from IPython.core.display import HTML 

from pandas.io.json import json_normalize

import folium

print('Lib ok')

from geopy.geocoders import Nominatim
import geocoder

Lib ok


In [28]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_London_boroughs')[0]
df.head()

,Borough,Inner,Status,Local authority,Political control,Headquarters,Area (sq mi),Population (2013 est)[1],Co-ordinates,Nr. in map
0,Barking and Dagenham [note 1],NaN,NaN,Barking and Dagenham London Borough Council,Labour,"Town Hall, 1 Town Square",13.93,194352,51°33′39″N 0°09′21″E﻿ / ﻿51.5607°N 0.1557°E,25
1,Barnet,NaN,NaN,Barnet London Borough Council,Conservative,"Barnet House, 2 Bristol Avenue, Colindale",33.49,369088,51°37′31″N 0°09′06″W﻿ / ﻿51.6252°N 0.1517°W,31
2,Bexley,NaN,NaN,Bexley London Borough Council,Conservative,"Civic Offices, 2 Watling Street",23.38,236687,51°27′18″N 0°09′02″E﻿ / ﻿51.4549°N 0.1505°E,23
3,Brent,NaN,NaN,Brent London Borough Council,Labour,"Brent Civic Centre, Engineers Way",16.70,317264,51°33′32″N 0°16′54″W﻿ / ﻿51.5588°N 0.2817°W,12
4,Bromley,NaN,NaN,Bromley London Borough Council,Conservative,"Civic Centre, Stockwell Close",57.97,317899,51°24′14″N 0°01′11″E﻿ / ﻿51.4039°N 0.0198°E,20


In [30]:

df.shape

(32, 10)

In [31]:
df = df.drop(columns="Inner")
df = df.drop(columns="Status")
df = df.drop(columns="Local authority")
df = df.drop(columns="Political control")
df = df.drop(columns="Headquarters")
df = df.drop(columns="Nr. in map")
df = df.drop(columns="Co-ordinates")
df.at[0, 'Borough'] = 'Barking and Dagenham'
df.at[11, 'Borough'] = 'Hammersmith and Fulham'
df.at[9, 'Borough'] = 'Greenwich'

In [32]:
def coordinates_hydrate(borough):
    coordinates = None 
    while(coordinates is None):
        g = geocoder.arcgis('{},London,UK'.format(borough))
        coordinates = g.latlng
        
    return coordinates

In [33]:
coordinates = [ coordinates_hydrate(borough) for borough in df["Borough"].tolist() ]
df_coordinates = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])

In [34]:
df_coordinates.head()

,Latitude,Longitude
0,51.543932,0.133157
1,51.627294,-0.253759
2,51.622832,-0.080656
3,51.609776,-0.194679
4,51.431820,-0.016566


In [35]:
df['Latitude'] = df_coordinates['Latitude']
df['Longitude'] = df_coordinates['Longitude']

In [36]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [13]:

map_london = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough in zip(df['Latitude'], df['Longitude'], df['Borough'],):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [68]:
CLIENT_ID = 'userrajeshkumar@gmail.com' # your Foursquare ID
CLIENT_SECRET = 'xxxxxxxx' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)
LIMIT = 100 
search_query = 'Hospital'

Your credentails:
CLIENT_ID: userrajeshkumar@gmail.com
CLIENT_SECRET:tunnababu


In [69]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
   
    search_query = 'Hospital'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, search_query, radius, LIMIT)
            
       
        results = requests.get(url).json()['response']['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Borough', 
                  'Borough Latitude', 
                  'Borough Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [70]:
london_venues = getNearbyVenues(names=df['Borough'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Barking and Dagenham


KeyError: 'groups'

In [71]:
print(london_venues.shape)

NameError: name 'london_venues' is not defined

In [64]:
london_venues.groupby('Borough').count()

NameError: name 'london_venues' is not defined

In [65]:
venue_map = london_venues 
venue_map['Latitude'] = london_venues['Venue Latitude']
venue_map['Longitude'] = london_venues['Venue Longitude']
venue_map['Venue'] = london_venues['Venue']

NameError: name 'london_venues' is not defined

In [66]:
address = 'London, UK'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London are 51.5073219, -0.1276474.


In [67]:
# create map of New York using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, Venue in zip(london_venues['Latitude'], london_venues['Longitude'], london_venues['Borough'],):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

NameError: name 'london_venues' is not defined